In [612]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import plotly.graph_objects as go

import random
import pandas as pd
import numpy as np

!pip install torchinfo
from torchinfo import summary

In [613]:
# Random Seed 고정 (학습 반복 시행 시에도 동일한 결과가 나오도록)

seed = 20250305

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [614]:
# line 을 보다 길게 표시 + 지수 표현이 아닌 원래 숫자 표현으로

torch.set_printoptions(linewidth=160, sci_mode=False)

**1. 데이터셋 로딩 및 데이터 분석**

In [615]:
# 데이터셋 로딩

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transform,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transform,
                                          download=True)


In [616]:
# 시간 절약을 위해, 학습 데이터에서 랜덤하게 일부 샘플만 추출

from torch.utils.data import Subset, DataLoader

NUM_TRAIN_SAMPLES = 5000
BATCH_SIZE = 32

subset_indices = random.sample(range(len(train_dataset)), NUM_TRAIN_SAMPLES)
train_subset = Subset(train_dataset, subset_indices)

train_loader = DataLoader(train_subset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

# 테스트 데이터셋은 학습 대상이 아니므로 그대로 이용
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False)

In [617]:
# 클래스 불균형 분석

# 학습 데이터
train_labels = torch.tensor([train_subset.dataset.targets[i] for i in subset_indices])
train_class_counts = torch.bincount(train_labels)
print(train_class_counts)

NUM_CLASSES = len(train_class_counts)

tensor([474, 555, 537, 545, 505, 422, 485, 492, 478, 507])


In [618]:
train_class_percentage = np.array(train_class_counts) * 100.0 / sum(train_class_counts)

train_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                                'count': train_class_counts,
                                'percentage (%)': train_class_percentage})

train_y_distrib

,class,count,percentage (%)
0,0,474,9.48
1,1,555,11.10
2,2,537,10.74
3,3,545,10.90
4,4,505,10.10
5,5,422,8.44
6,6,485,9.70
7,7,492,9.84
8,8,478,9.56
9,9,507,10.14


In [619]:
# 테스트 데이터
test_labels = test_loader.dataset.targets
test_class_counts = torch.bincount(test_labels)
print(test_class_counts)

tensor([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009])


In [620]:
test_class_percentage = np.array(test_class_counts) * 100.0 / sum(test_class_counts)

test_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                               'count': test_class_counts,
                               'percentage (%)': test_class_percentage})

test_y_distrib

,class,count,percentage (%)
0,0,980,9.80
1,1,1135,11.35
2,2,1032,10.32
3,3,1010,10.10
4,4,982,9.82
5,5,892,8.92
6,6,958,9.58
7,7,1028,10.28
8,8,974,9.74
9,9,1009,10.09


**2. CNN 모델 정의**

In [621]:
# CNN 모델 정의

class CNN(nn.Module):

    def __init__(self, num_classes=10, final_activation=nn.Softmax()):
        super(CNN, self).__init__()

        # Conv
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU()
        )
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )

        # Fully Connected
        self.fc1 = nn.Sequential(
            nn.Linear(64 * 4 * 4, 64),
            nn.Sigmoid()
        )

        if final_activation is not None:
            self.fc_final = nn.Sequential(
                nn.Linear(64, num_classes),
                final_activation
            )
        else:
            self.fc_final = nn.Linear(64, num_classes)

    def forward(self, x):

        # Conv
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)

        x = x.view(-1, 64 * 4 * 4)

        # Fully Connected
        x = self.fc1(x)
        x = self.fc_final(x)

        return x

In [622]:
# 모델 구조 출력

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

print(summary(model, input_size=(BATCH_SIZE, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [32, 10]                  --
├─Sequential: 1-1                        [32, 32, 28, 28]          --
│    └─Conv2d: 2-1                       [32, 32, 28, 28]          320
│    └─ReLU: 2-2                         [32, 32, 28, 28]          --
├─MaxPool2d: 1-2                         [32, 32, 14, 14]          --
├─Sequential: 1-3                        [32, 64, 12, 12]          --
│    └─Conv2d: 2-3                       [32, 64, 12, 12]          18,496
│    └─ReLU: 2-4                         [32, 64, 12, 12]          --
├─MaxPool2d: 1-4                         [32, 64, 6, 6]            --
├─Sequential: 1-5                        [32, 64, 4, 4]            --
│    └─Conv2d: 2-5                       [32, 64, 4, 4]            36,928
│    └─ReLU: 2-6                         [32, 64, 4, 4]            --
├─Sequential: 1-6                        [32, 64]                  --
│    └

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1844: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return inner()


**3. 데이터셋 분리**

* Train Data -> Train Data + Valid Data

In [623]:
# 데이터셋 분리

from torch.utils.data import random_split

# 샘플 수
num_train = 2000
num_valid = 3000

assert NUM_TRAIN_SAMPLES == num_train + num_valid

# 데이터셋 분리
train_dataset, valid_dataset =\
    random_split(train_subset, [num_train, num_valid])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

**4. 학습 실시 함수**

In [624]:
MAX_EPOCHS = 65536
EARLY_STOPPING_ROUNDS = 10  # Early Stopping Patience (epochs)

In [625]:
from sklearn.metrics import accuracy_score
from copy import deepcopy

In [626]:
# 모델 학습 실시

# args :
# - model           : 학습할 모델
# - train_loader    : Training Data Loader
# - train_loss_list : 각 epoch 에서의 train loss 기록

# returns :
# - train_loss : 모델의 Train Loss

def run_train(model, train_loader, train_loss_list):
    model.train()
    train_loss = 0.0
    cnt = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # train 실시
        model.optimizer.zero_grad()
        outputs = model(images)

        # Regression or Prob. Prediction 일 때 처리
        if model.task_type in ['regression', 'prob_prediction']:
            outputs = torch.flatten(outputs).float()
            labels = labels.float()

        # Classification - Multi Label 일 때 처리
        elif model.multi_label:
            pass

        # Classification - Single Label 일 때
        # Loss Function 이 Binary Cross Entropy / Mean Squared Error 인 경우 output shape 변경 처리
        elif model.loss_function.__class__.__name__ in ['BCELoss', 'BCEWithLogitsLoss', 'MSELoss']:
            labels = torch.nn.functional.one_hot(labels,
                                                 num_classes=model.num_classes)
            labels = labels.float()

        # Loss 처리
        loss = model.loss_function(outputs, labels)
        loss.backward()
        model.optimizer.step()

        train_loss += loss.item()
        cnt += 1

    train_loss_list.append(train_loss / len(train_loader))
    return train_loss_list[-1]

In [627]:
# 모델 validation 실시

# args :
# - model                  : validation 할 모델
# - valid_loader           : Validation Data Loader
# - normalized_output      : dataset 의 출력값이 normalize 되었는지 여부
# - print_first_batch_info : 1번째 batch 에서 output, label 을 출력할지 여부

# returns :
# - accuracy : 모델의 validation 정확도 (Classification or Prob. Prediction 인 경우)
# - mse      : 모델의 validation MSE (Regression 인 경우)

def run_validation(model, valid_loader, normalized_output=False, print_first_batch_info=False):
    model.eval()
    correct, total = 0, 0
    squared_error_sum = 0.0
    print_output_and_label = True

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # 정규화 처리
            if normalized_output:
                outputs = outputs * model.train_output_std + model.train_output_mean
                labels = labels * model.train_output_std + model.train_output_mean

            # 첫번째 batch 의 output 및 label 출력
            if print_first_batch_info and print_output_and_label:
                print(f'[ outputs ]\n{outputs}\n[ labels ]\n{labels}')
                print_output_and_label = False

            # validation 실시 및 정확도 측정
            total += labels.size(0)

            # For Regression
            if model.task_type == 'regression':
                outputs = torch.flatten(outputs).float()
                labels = torch.flatten(labels).float()
                squared_error_sum += nn.MSELoss()(outputs, labels) * labels.size(0)

            # For Probability Prediction
            elif model.task_type == 'prob_prediction':
                outputs_hard = torch.round(outputs)
                outputs_hard = torch.flatten(outputs_hard)
                correct += (outputs_hard == labels).sum().item()

            # For Multi Label Classification
            # (각 sample 당 존재하는 num_classes 개의 0 ~ 1 값을 prediction 으로 간주하고,
            #  모든 sample 에 대한 전체 prediction 중 정답 prediction 의 비율로 정확도 측정)
            elif model.multi_label:
                outputs_hard = torch.round(outputs)
                correct += (outputs_hard == labels).sum().item() / model.num_classes

            # For Single Label Classification
            else:
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()

    if model.task_type == 'regression':
        mse = squared_error_sum / total
        return mse

    else:
        accuracy = correct / total
        return accuracy

In [628]:
# metric 이름 반환

def get_metric_name(task_type):
    if task_type == 'regression':
        return 'MSE'
    else:
        return 'Accuracy'

In [629]:
# Regression 정규화 시 학습 데이터 출력값의 평균, 표준편차 정보 저장

# args :
# - model        : 학습할 모델
# - train_loader : Training Data Loader

# returns :
# - model.train_output_mean 에 Training Data output 의 평균 저장
# - model.train_output_std 에 Training Data output 의 표준편차 저장

def store_train_output_mean_std(model, train_loader):
    output_list = []

    for inputs, labels in train_loader:
        output_list += list(labels.cpu().numpy())

    model.train_output_mean = np.mean(output_list)
    model.train_output_std = np.std(output_list)

In [630]:
from torch.utils.data import Dataset

class NormalizedDataset(Dataset):
    def __init__(self, dataset, mean, std):
        self.dataset = dataset
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        y_standardized = (y - self.mean) / self.std
        return x, y_standardized


# DataLoader 의 모든 출력값을 Normalize 처리

# args :
# - data_loader : 모든 출력값을 Normalize 처리할 DataLoader
# - mean        : 평균값
# - std         : 표준편차
# - shuffle     : shuffle 여부 (train 에서만 True)

def generate_normalized_dataloader(data_loader, mean, std, shuffle):
    normalized_dataset = NormalizedDataset(data_loader.dataset, mean, std)
    new_dataloader = DataLoader(normalized_dataset, batch_size=BATCH_SIZE, shuffle=shuffle)

    return new_dataloader

In [631]:
# 모델 학습 및 validation 전체 프로세스

# args :
# - model        : 학습할 모델
# - train_loader : Training Data Loader
# - valid_loader : 각 epoch 마다 validation 할 Valid Data Loader
# - test_loader  : 최종적으로 성능을 평가할 Test Data Loader
# - verbose      : 학습 중 프로세스 출력 여부

# returns :
# - final_metric     : 해당 하이퍼파라미터 조합에 대한 최종 성능지표 값 (valid metric 이 가장 좋았던 epoch 의 모델로 측정)
# - best_epoch_model : valid metric 이 가장 좋았던 epoch 에서 생성된 모델

def run_model_common(model, train_loader, valid_loader, test_loader, verbose=False):

    train_loss_list = []          # train loss
    valid_metric_list = []        # valid metric (Accuracy for Classification, MSE for Regression)
    best_valid_metric = None      # best validation accuracy
    best_valid_metric_epoch = -1  # valid metric 이 가장 좋았던 epoch
    best_epoch_model = None       # valid metric 이 가장 좋았던 epoch 의 모델

    metric_name = get_metric_name(model.task_type)
    do_normalize = model.additional_options.get('normalization', False) is True

    # 평균 및 표준편차 저장 및 표준화된 Data Loader 생성
    if do_normalize:
        store_train_output_mean_std(model, train_loader)
        print(f'train output mean: {model.train_output_mean}, std: {model.train_output_std}')

        train_loader_norm = generate_normalized_dataloader(data_loader=train_loader,
                                                           mean=model.train_output_mean,
                                                           std=model.train_output_std,
                                                           shuffle=True)

        valid_loader_norm = generate_normalized_dataloader(data_loader=valid_loader,
                                                           mean=model.train_output_mean,
                                                           std=model.train_output_std,
                                                           shuffle=False)

        test_loader_norm = generate_normalized_dataloader(data_loader=test_loader,
                                                          mean=model.train_output_mean,
                                                          std=model.train_output_std,
                                                          shuffle=False)

        train_loader_ = train_loader_norm
        valid_loader_ = valid_loader_norm
        test_loader_ = test_loader_norm

    else:
        train_loader_ = train_loader
        valid_loader_ = valid_loader
        test_loader_ = test_loader

    # 1. 학습 실시
    for epoch in range(MAX_EPOCHS):

        # 1-1. train model
        train_loss = run_train(model, train_loader_, train_loss_list)

        # 1-2. validate model (with EPOCH VALID SET)
        epoch_metric = run_validation(model=model,
                                      valid_loader=valid_loader_,
                                      normalized_output=do_normalize)

        valid_metric_list.append(epoch_metric)

        # 1-3. Early Stopping 처리 (overfitting 방지)
        if epoch == 0:
            improved = True  # 1st epoch 에서는 항상 improve 되었다고 가정
        elif model.task_type == 'regression':
            improved = epoch_metric < best_valid_metric
        else:
            improved = epoch_metric > best_valid_metric

        if improved:
            best_valid_metric = epoch_metric
            best_valid_metric_epoch = epoch

            best_epoch_model = CNN(num_classes=model.num_classes,
                                   final_activation=model.final_activation).to(device)

            best_epoch_model.multi_label = model.multi_label
            best_epoch_model.num_classes = model.num_classes
            best_epoch_model.task_type = model.task_type
            best_epoch_model.additional_options = model.additional_options

            if do_normalize:
                best_epoch_model.train_output_mean = model.train_output_mean
                best_epoch_model.train_output_std = model.train_output_std

            best_epoch_model.load_state_dict(model.state_dict())

            if verbose:
                print('best model updated')

        if epoch - best_valid_metric_epoch >= EARLY_STOPPING_ROUNDS:
            total_epochs = epoch
            break

        # 1-4. 결과 출력
        if verbose:
            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid {metric_name}: {epoch_metric:.4f}")

    # check best-epoch model correctly loaded
    checked_metric = run_validation(model=best_epoch_model,
                                    valid_loader=valid_loader_,
                                    normalized_output=do_normalize)

    if verbose:
        print(f"Best Epoch: {best_valid_metric_epoch}, Best Valid {metric_name}: {best_valid_metric}")
        print(f"Valid {metric_name} (with VALID set) on Loaded Best Model: {checked_metric}")

    assert abs(best_valid_metric - checked_metric) < 1e-8

    # 2. validate best-epoch model (with TEST SET)
    final_metric = run_validation(model=best_epoch_model,
                                  valid_loader=test_loader_,
                                  normalized_output=do_normalize,
                                  print_first_batch_info=True)

    if verbose:
        print(f"Final {metric_name} (with TEST set) on Loaded Best Model: {final_metric}")

    return final_metric, best_epoch_model, total_epochs

In [632]:
print(device)

cuda


**5. 실험 실시**

In [633]:
# 실험 실시

# args:
# - loss_function      : 모델 학습에 사용할 Loss Function
# - train_loader       : Training Data Loader
# - valid_loader       : 각 epoch 마다 validation 할 Valid Data Loader
# - test_loader        : 최종적으로 성능을 평가할 Test Data Loader
# - num_classes        : 모델이 분류할 class 개수 (또는 output value 개수)
# - task_type          : 'classification', 'regression' or 'prob_prediction'
# - final_activation   : 모델의 최종 Layer 의 Activation Function
# - multi_label        : Multi-Label 여부
# - additional_options : 모델 configuration 추가 옵션

# returns:
# - final_metric     : Test dataset 성능지표 값
# - best_epoch_model : Valid dataset 성능지표 값이 가장 높은 모델

def run_experiment(loss_function, train_loader, valid_loader, test_loader,
                   num_classes=10, task_type='classification',
                   final_activation=nn.Softmax(), multi_label=False,
                   additional_options={}):

    # define and run model
    model = CNN(num_classes, final_activation).to(device)

    # model configurations
    model.num_classes = num_classes
    model.final_activation = final_activation
    model.loss_function = loss_function
    model.optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    model.multi_label = multi_label
    model.task_type = task_type
    model.additional_options = additional_options

#    print(summary(model, input_size=(BATCH_SIZE, 1, 28, 28)))

    final_metric, best_epoch_model, total_epochs = run_model_common(model,
                                                                    train_loader,
                                                                    valid_loader,
                                                                    test_loader,
                                                                    verbose=True)

    metric_name = get_metric_name(model.task_type)
    print(f"{metric_name}: {final_metric:.4f}, Total Epochs: {total_epochs}")

    return final_metric, best_epoch_model

In [634]:
# 실험 함수의 정상 동작 여부 확인

base_loss_function = nn.CrossEntropyLoss()
_, _ = run_experiment(base_loss_function, train_loader, valid_loader, test_loader)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 2.1659, Valid Accuracy: 0.5947
best model updated
Epoch 2, Train Loss: 1.9115, Valid Accuracy: 0.7897
best model updated
Epoch 3, Train Loss: 1.7405, Valid Accuracy: 0.8587
best model updated
Epoch 4, Train Loss: 1.6519, Valid Accuracy: 0.8713
Epoch 5, Train Loss: 1.6115, Valid Accuracy: 0.8687
best model updated
Epoch 6, Train Loss: 1.5925, Valid Accuracy: 0.8853
best model updated
Epoch 7, Train Loss: 1.5587, Valid Accuracy: 0.9593
best model updated
Epoch 8, Train Loss: 1.5161, Valid Accuracy: 0.9600
Epoch 9, Train Loss: 1.5015, Valid Accuracy: 0.9560
best model updated
Epoch 10, Train Loss: 1.4907, Valid Accuracy: 0.9683
best model updated
Epoch 11, Train Loss: 1.4811, Valid Accuracy: 0.9693
Epoch 12, Train Loss: 1.4772, Valid Accuracy: 0.9690
Epoch 13, Train Loss: 1.4747, Valid Accuracy: 0.9693
Epoch 14, Train Loss: 1.4725, Valid Accuracy: 0.9683
best model updated
Epoch 15, Train Loss: 1.4712, Valid Accuracy: 0.9710
Epoch 16, Train Loss: 1.

5-1. Regression

In [635]:
# Regression 용으로 변경

from torch.utils.data import Dataset

class MNIST_reg(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.regression_map = {1: 1.0,
                               0: 2.0, 2: 2.0, 3: 2.0, 7: 2.0,
                               5: 3.0, 6: 3.0, 9: 3.0,
                               8: 4.0,
                               4: 6.0}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        new_label = self.regression_map[label]
        return image, new_label

In [636]:
# 데이터셋 클래스 분포 분석 함수

def analyze_class_distribution(data_loader):
    targets = [data_loader.dataset[i][1] for i in range(len(data_loader.dataset))]

    class_list, class_counts = np.unique(targets, return_counts=True)
    class_percentage = np.array(class_counts) * 100.0 / sum(class_counts)

    class_distrib = pd.DataFrame({'class': class_list,
                                  'count': class_counts,
                                  'percentage (%)': class_percentage})

    return class_distrib

In [637]:
# Regression 용 Data Loader

train_loader_reg = DataLoader(MNIST_reg(train_loader.dataset),
                              batch_size=BATCH_SIZE,
                              shuffle=True)

valid_loader_reg = DataLoader(MNIST_reg(valid_loader.dataset),
                              batch_size=BATCH_SIZE,
                              shuffle=False)

test_loader_reg = DataLoader(MNIST_reg(test_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=False)

In [638]:
analyze_class_distribution(train_loader_reg)

,class,count,percentage (%)
0,1.0,239,11.95
1,2.0,795,39.75
2,3.0,585,29.25
3,4.0,181,9.05
4,6.0,200,10.00


In [639]:
analyze_class_distribution(valid_loader_reg)

,class,count,percentage (%)
0,1.0,316,10.533333
1,2.0,1253,41.766667
2,3.0,829,27.633333
3,4.0,297,9.900000
4,6.0,305,10.166667


In [640]:
analyze_class_distribution(test_loader_reg)

,class,count,percentage (%)
0,1.0,1135,11.35
1,2.0,4050,40.50
2,3.0,2859,28.59
3,4.0,974,9.74
4,6.0,982,9.82


In [641]:
# Regression (정규화 미 적용 시) 실험 진행

_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_reg,
                                     valid_loader=valid_loader_reg,
                                     test_loader=test_loader_reg,
                                     num_classes=1,
                                     task_type='regression',
                                     final_activation=None)

best model updated
Epoch 1, Train Loss: 2.3883, Valid MSE: 1.8027
Epoch 2, Train Loss: 1.8077, Valid MSE: 1.8028
Epoch 3, Train Loss: 1.8006, Valid MSE: 1.8028
Epoch 4, Train Loss: 1.8071, Valid MSE: 1.8027
Epoch 5, Train Loss: 1.8075, Valid MSE: 1.8102
Epoch 6, Train Loss: 1.8106, Valid MSE: 1.8034
Epoch 7, Train Loss: 1.8159, Valid MSE: 1.8048
Epoch 8, Train Loss: 1.8135, Valid MSE: 1.8061
best model updated
Epoch 9, Train Loss: 1.8004, Valid MSE: 1.8027
Epoch 10, Train Loss: 1.8140, Valid MSE: 1.8028
Epoch 11, Train Loss: 1.8120, Valid MSE: 1.8055
Epoch 12, Train Loss: 1.8084, Valid MSE: 1.8051
Epoch 13, Train Loss: 1.8109, Valid MSE: 1.8071
Epoch 14, Train Loss: 1.8088, Valid MSE: 1.8044
Epoch 15, Train Loss: 1.7973, Valid MSE: 1.8027
Epoch 16, Train Loss: 1.7970, Valid MSE: 1.8043
Epoch 17, Train Loss: 1.8116, Valid MSE: 1.8028
Epoch 18, Train Loss: 1.8024, Valid MSE: 1.8089
Best Epoch: 8, Best Valid MSE: 1.8026864528656006
Valid MSE (with VALID set) on Loaded Best Model: 1.802686

In [642]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_reg,
                                     valid_loader=valid_loader_reg,
                                     test_loader=test_loader_reg,
                                     num_classes=1,
                                     task_type='regression',
                                     final_activation=nn.ReLU())

best model updated
Epoch 1, Train Loss: 2.0741, Valid MSE: 1.8085
best model updated
Epoch 2, Train Loss: 1.8065, Valid MSE: 1.8053
best model updated
Epoch 3, Train Loss: 1.8202, Valid MSE: 1.8027
Epoch 4, Train Loss: 1.8096, Valid MSE: 1.8032
Epoch 5, Train Loss: 1.8009, Valid MSE: 1.8029
Epoch 6, Train Loss: 1.8085, Valid MSE: 1.8073
Epoch 7, Train Loss: 1.8020, Valid MSE: 1.8126
Epoch 8, Train Loss: 1.8128, Valid MSE: 1.8058
Epoch 9, Train Loss: 1.8089, Valid MSE: 1.8057
Epoch 10, Train Loss: 1.8100, Valid MSE: 1.8059
Epoch 11, Train Loss: 1.8112, Valid MSE: 1.8060
best model updated
Epoch 12, Train Loss: 1.8067, Valid MSE: 1.8027
Epoch 13, Train Loss: 1.8110, Valid MSE: 1.8034
Epoch 14, Train Loss: 1.7994, Valid MSE: 1.8067
Epoch 15, Train Loss: 1.8127, Valid MSE: 1.8104
Epoch 16, Train Loss: 1.8195, Valid MSE: 1.8036
Epoch 17, Train Loss: 1.8068, Valid MSE: 1.8044
Epoch 18, Train Loss: 1.8112, Valid MSE: 1.8027
Epoch 19, Train Loss: 1.8052, Valid MSE: 1.8042
Epoch 20, Train Loss:

In [643]:
# Regression (정규화 적용 시) 실험 진행

_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_reg,
                                     valid_loader=valid_loader_reg,
                                     test_loader=test_loader_reg,
                                     num_classes=1,
                                     task_type='regression',
                                     additional_options={'normalization': True},
                                     final_activation=None)

train output mean: 2.754, std: 1.343682998329591
best model updated
Epoch 1, Train Loss: 0.5083, Valid MSE: 0.4907
best model updated
Epoch 2, Train Loss: 0.2222, Valid MSE: 0.3237
best model updated
Epoch 3, Train Loss: 0.1529, Valid MSE: 0.2417
best model updated
Epoch 4, Train Loss: 0.1087, Valid MSE: 0.1808
best model updated
Epoch 5, Train Loss: 0.0807, Valid MSE: 0.1762
best model updated
Epoch 6, Train Loss: 0.0793, Valid MSE: 0.1631
best model updated
Epoch 7, Train Loss: 0.0512, Valid MSE: 0.1284
Epoch 8, Train Loss: 0.0392, Valid MSE: 0.1480
best model updated
Epoch 9, Train Loss: 0.0353, Valid MSE: 0.1232
best model updated
Epoch 10, Train Loss: 0.0283, Valid MSE: 0.1131
best model updated
Epoch 11, Train Loss: 0.0224, Valid MSE: 0.0989
best model updated
Epoch 12, Train Loss: 0.0167, Valid MSE: 0.0978
best model updated
Epoch 13, Train Loss: 0.0113, Valid MSE: 0.0963
Epoch 14, Train Loss: 0.0092, Valid MSE: 0.0998
Epoch 15, Train Loss: 0.0081, Valid MSE: 0.1023
best model u

In [644]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_reg,
                                     valid_loader=valid_loader_reg,
                                     test_loader=test_loader_reg,
                                     num_classes=1,
                                     task_type='regression',
                                     additional_options={'normalization': True},
                                     final_activation=nn.ReLU())

train output mean: 2.754, std: 1.343682998329591
best model updated
Epoch 1, Train Loss: 0.7099, Valid MSE: 0.8678
best model updated
Epoch 2, Train Loss: 0.5134, Valid MSE: 0.8275
best model updated
Epoch 3, Train Loss: 0.4855, Valid MSE: 0.7951
best model updated
Epoch 4, Train Loss: 0.4627, Valid MSE: 0.7860
best model updated
Epoch 5, Train Loss: 0.4422, Valid MSE: 0.7830
Epoch 6, Train Loss: 0.4393, Valid MSE: 0.7939
Epoch 7, Train Loss: 0.4311, Valid MSE: 0.7981
best model updated
Epoch 8, Train Loss: 0.4231, Valid MSE: 0.7701
Epoch 9, Train Loss: 0.4232, Valid MSE: 0.8232
Epoch 10, Train Loss: 0.4208, Valid MSE: 0.7706
Epoch 11, Train Loss: 0.4217, Valid MSE: 0.7759
Epoch 12, Train Loss: 0.4166, Valid MSE: 0.7847
Epoch 13, Train Loss: 0.4148, Valid MSE: 0.7857
Epoch 14, Train Loss: 0.4147, Valid MSE: 0.7875
Epoch 15, Train Loss: 0.4166, Valid MSE: 0.7727
best model updated
Epoch 16, Train Loss: 0.4036, Valid MSE: 0.7339
best model updated
Epoch 17, Train Loss: 0.3843, Valid MSE:

5-2. Probability Prediction

In [645]:
# Probability Prediction 용으로 변경

from torch.utils.data import Dataset

class MNIST_pp(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.class_map = {0: 1.0, 3: 1.0, 6: 1.0, 8: 1.0, 9: 1.0,
                          1: 0.0, 2: 0.0, 4: 0.0, 5: 0.0, 7: 0.0}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        new_label = self.class_map[label]
        return image, new_label

In [646]:
# Probability Prediction 용 Data Loader

train_loader_pp = DataLoader(MNIST_pp(train_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=True)

valid_loader_pp = DataLoader(MNIST_pp(valid_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=False)

test_loader_pp = DataLoader(MNIST_pp(test_loader.dataset),
                            batch_size=BATCH_SIZE,
                            shuffle=False)

In [647]:
analyze_class_distribution(train_loader_pp)

,class,count,percentage (%)
0,0.0,1007,50.35
1,1.0,993,49.65


In [648]:
analyze_class_distribution(valid_loader_pp)

,class,count,percentage (%)
0,0.0,1504,50.133333
1,1.0,1496,49.866667


In [649]:
analyze_class_distribution(test_loader_pp)

,class,count,percentage (%)
0,0.0,5069,50.69
1,1.0,4931,49.31


In [650]:
# Probability Prediction 실험 진행

_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.6269, Valid Accuracy: 0.7837
best model updated
Epoch 2, Train Loss: 0.3965, Valid Accuracy: 0.8087
best model updated
Epoch 3, Train Loss: 0.2787, Valid Accuracy: 0.9080
best model updated
Epoch 4, Train Loss: 0.1931, Valid Accuracy: 0.9267
best model updated
Epoch 5, Train Loss: 0.1342, Valid Accuracy: 0.9480
Epoch 6, Train Loss: 0.0920, Valid Accuracy: 0.9443
best model updated
Epoch 7, Train Loss: 0.0783, Valid Accuracy: 0.9560
best model updated
Epoch 8, Train Loss: 0.0571, Valid Accuracy: 0.9603
Epoch 9, Train Loss: 0.0345, Valid Accuracy: 0.9593
Epoch 10, Train Loss: 0.0497, Valid Accuracy: 0.9590
Epoch 11, Train Loss: 0.0199, Valid Accuracy: 0.9600
best model updated
Epoch 12, Train Loss: 0.0139, Valid Accuracy: 0.9633
Epoch 13, Train Loss: 0.0194, Valid Accuracy: 0.9600
best model updated
Epoch 14, Train Loss: 0.0142, Valid Accuracy: 0.9643
Epoch 15, Train Loss: 0.0058, Valid Accuracy: 0.9637
best model updated
Epoch 16, Train Loss: 0.

In [651]:
_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=nn.Softmax())

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 50.4464, Valid Accuracy: 0.4987
Epoch 2, Train Loss: 50.2976, Valid Accuracy: 0.4987
Epoch 3, Train Loss: 50.2976, Valid Accuracy: 0.4987
Epoch 4, Train Loss: 50.2976, Valid Accuracy: 0.4987
Epoch 5, Train Loss: 50.2976, Valid Accuracy: 0.4987
Epoch 6, Train Loss: 50.1984, Valid Accuracy: 0.4987
Epoch 7, Train Loss: 50.3968, Valid Accuracy: 0.4987
Epoch 8, Train Loss: 50.3968, Valid Accuracy: 0.4987
Epoch 9, Train Loss: 50.3472, Valid Accuracy: 0.4987
Epoch 10, Train Loss: 50.3472, Valid Accuracy: 0.4987
Best Epoch: 0, Best Valid Accuracy: 0.49866666666666665
Valid Accuracy (with VALID set) on Loaded Best Model: 0.49866666666666665
[ outputs ]
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],


In [652]:
_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 54.5496, Valid Accuracy: 0.4987
Epoch 2, Train Loss: 54.5496, Valid Accuracy: 0.4987
Epoch 3, Train Loss: 54.5276, Valid Accuracy: 0.4987
Epoch 4, Train Loss: 54.5716, Valid Accuracy: 0.4987
Epoch 5, Train Loss: 54.5056, Valid Accuracy: 0.4987
Epoch 6, Train Loss: 54.5716, Valid Accuracy: 0.4987
Epoch 7, Train Loss: 54.5496, Valid Accuracy: 0.4987
Epoch 8, Train Loss: 54.5606, Valid Accuracy: 0.4987
Epoch 9, Train Loss: 54.5496, Valid Accuracy: 0.4987
Epoch 10, Train Loss: 54.5056, Valid Accuracy: 0.4987
Best Epoch: 0, Best Valid Accuracy: 0.49866666666666665
Valid Accuracy (with VALID set) on Loaded Best Model: 0.49866666666666665
[ outputs ]
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],


5-3. Binary Classification (2 outputs)

In [653]:
# Binary Classification 용으로 변경

from torch.utils.data import Dataset

class MNIST_bc(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.class_map = {0: 1, 3: 1, 6: 1, 8: 1, 9: 1,
                          1: 0, 2: 0, 4: 0, 5: 0, 7: 0}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        new_label = self.class_map[label]
        return image, new_label

In [654]:
# Binary Classification 용 Data Loader

train_loader_bc = DataLoader(MNIST_bc(train_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=True)

valid_loader_bc = DataLoader(MNIST_bc(valid_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=False)

test_loader_bc = DataLoader(MNIST_bc(test_loader.dataset),
                            batch_size=BATCH_SIZE,
                            shuffle=False)

In [655]:
analyze_class_distribution(train_loader_bc)

,class,count,percentage (%)
0,0,1007,50.35
1,1,993,49.65


In [656]:
analyze_class_distribution(valid_loader_bc)

,class,count,percentage (%)
0,0,1504,50.133333
1,1,1496,49.866667


In [657]:
analyze_class_distribution(test_loader_bc)

,class,count,percentage (%)
0,0,5069,50.69
1,1,4931,49.31


In [658]:
# Binary Classification 을 Class 2개 기준의 Categorical Cross-Entropy 를 이용하여 실험 진행

_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Softmax())

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 0.5920, Valid Accuracy: 0.8007
best model updated
Epoch 2, Train Loss: 0.4776, Valid Accuracy: 0.8477
best model updated
Epoch 3, Train Loss: 0.4521, Valid Accuracy: 0.8580
best model updated
Epoch 4, Train Loss: 0.4344, Valid Accuracy: 0.8653
best model updated
Epoch 5, Train Loss: 0.4188, Valid Accuracy: 0.8893
best model updated
Epoch 6, Train Loss: 0.4039, Valid Accuracy: 0.8987
best model updated
Epoch 7, Train Loss: 0.3807, Valid Accuracy: 0.9327
best model updated
Epoch 8, Train Loss: 0.3549, Valid Accuracy: 0.9430
best model updated
Epoch 9, Train Loss: 0.3503, Valid Accuracy: 0.9553
Epoch 10, Train Loss: 0.3395, Valid Accuracy: 0.9553
Epoch 11, Train Loss: 0.3384, Valid Accuracy: 0.9507
Epoch 12, Train Loss: 0.3316, Valid Accuracy: 0.9493
best model updated
Epoch 13, Train Loss: 0.3317, Valid Accuracy: 0.9597
best model updated
Epoch 14, Train Loss: 0.3279, Valid Accuracy: 0.9607
Epoch 15, Train Loss: 0.3234, Valid Accuracy: 0.9603
best 

In [659]:
# 모델의 최종 출력값은 Sigmoid 함수를 통해 0 ~ 1 로 변환된 값임이 보장됨
# -> nn.BCEWithLogitsLoss() 사용 불필요

_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.5798, Valid Accuracy: 0.7787
best model updated
Epoch 2, Train Loss: 0.3950, Valid Accuracy: 0.8577
best model updated
Epoch 3, Train Loss: 0.2895, Valid Accuracy: 0.8717
best model updated
Epoch 4, Train Loss: 0.1944, Valid Accuracy: 0.9363
best model updated
Epoch 5, Train Loss: 0.1270, Valid Accuracy: 0.9520
best model updated
Epoch 6, Train Loss: 0.0809, Valid Accuracy: 0.9590
best model updated
Epoch 7, Train Loss: 0.0512, Valid Accuracy: 0.9593
best model updated
Epoch 8, Train Loss: 0.0482, Valid Accuracy: 0.9630
Epoch 9, Train Loss: 0.0282, Valid Accuracy: 0.9623
best model updated
Epoch 10, Train Loss: 0.0253, Valid Accuracy: 0.9653
Epoch 11, Train Loss: 0.0178, Valid Accuracy: 0.9613
Epoch 12, Train Loss: 0.0120, Valid Accuracy: 0.9647
best model updated
Epoch 13, Train Loss: 0.0064, Valid Accuracy: 0.9687
Epoch 14, Train Loss: 0.0037, Valid Accuracy: 0.9687
best model updated
Epoch 15, Train Loss: 0.0027, Valid Accuracy: 0.9690
Epoch

In [660]:
_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.6205, Valid Accuracy: 0.7577
best model updated
Epoch 2, Train Loss: 0.5062, Valid Accuracy: 0.8360
best model updated
Epoch 3, Train Loss: 0.4690, Valid Accuracy: 0.8643
best model updated
Epoch 4, Train Loss: 0.4319, Valid Accuracy: 0.8797
best model updated
Epoch 5, Train Loss: 0.4132, Valid Accuracy: 0.8953
best model updated
Epoch 6, Train Loss: 0.3899, Valid Accuracy: 0.8967
best model updated
Epoch 7, Train Loss: 0.3734, Valid Accuracy: 0.9183
best model updated
Epoch 8, Train Loss: 0.3638, Valid Accuracy: 0.9247
best model updated
Epoch 9, Train Loss: 0.3545, Valid Accuracy: 0.9427
best model updated
Epoch 10, Train Loss: 0.3465, Valid Accuracy: 0.9453
best model updated
Epoch 11, Train Loss: 0.3412, Valid Accuracy: 0.9540
Epoch 12, Train Loss: 0.3371, Valid Accuracy: 0.9470
Epoch 13, Train Loss: 0.3438, Valid Accuracy: 0.9280
best model updated
Epoch 14, Train Loss: 0.3373, Valid Accuracy: 0.9597
best model updated
Epoch 15, Train Loss

5-4. Multi-Class Classification

In [661]:
# Multi-Class Classification (총 10개 확률 output)

_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 2.1520, Valid Accuracy: 0.6580
best model updated
Epoch 2, Train Loss: 1.8616, Valid Accuracy: 0.7677
best model updated
Epoch 3, Train Loss: 1.7415, Valid Accuracy: 0.7783
best model updated
Epoch 4, Train Loss: 1.7011, Valid Accuracy: 0.7877
best model updated
Epoch 5, Train Loss: 1.6688, Valid Accuracy: 0.8543
best model updated
Epoch 6, Train Loss: 1.6148, Valid Accuracy: 0.9343
best model updated
Epoch 7, Train Loss: 1.5494, Valid Accuracy: 0.9567
best model updated
Epoch 8, Train Loss: 1.5161, Valid Accuracy: 0.9620
Epoch 9, Train Loss: 1.4972, Valid Accuracy: 0.9603
best model updated
Epoch 10, Train Loss: 1.4894, Valid Accuracy: 0.9643
best model updated
Epoch 11, Train Loss: 1.4837, Valid Accuracy: 0.9670
best model updated
Epoch 12, Train Loss: 1.4814, Valid Accuracy: 0.9680
best model updated
Epoch 13, Train Loss: 1.4755, Valid Accuracy: 0.9687
best model updated
Epoch 14, Train Loss: 1.4726, Valid Accuracy: 0.9690
best model updated
E

In [662]:
# 모델의 최종 출력값은 Sigmoid 함수를 통해 0 ~ 1 로 변환된 값임이 보장됨
# -> nn.BCEWithLogitsLoss() 사용 불필요

_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.3976, Valid Accuracy: 0.0830
best model updated
Epoch 2, Train Loss: 0.3264, Valid Accuracy: 0.1053
Epoch 3, Train Loss: 0.3250, Valid Accuracy: 0.1053
Epoch 4, Train Loss: 0.3251, Valid Accuracy: 0.1053
Epoch 5, Train Loss: 0.3252, Valid Accuracy: 0.1053
Epoch 6, Train Loss: 0.3250, Valid Accuracy: 0.1053
Epoch 7, Train Loss: 0.3251, Valid Accuracy: 0.1053
Epoch 8, Train Loss: 0.3251, Valid Accuracy: 0.1053
Epoch 9, Train Loss: 0.3252, Valid Accuracy: 0.1053
Epoch 10, Train Loss: 0.3250, Valid Accuracy: 0.1053
Epoch 11, Train Loss: 0.3251, Valid Accuracy: 0.1053
Best Epoch: 1, Best Valid Accuracy: 0.10533333333333333
Valid Accuracy (with VALID set) on Loaded Best Model: 0.10533333333333333
[ outputs ]
tensor([[0.0988, 0.1228, 0.0956, 0.1057, 0.1030, 0.0973, 0.1058, 0.0981, 0.0964, 0.1018],
        [0.0988, 0.1228, 0.0956, 0.1057, 0.1030, 0.0973, 0.1058, 0.0981, 0.0964, 0.1018],
        [0.0988, 0.1228, 0.0956, 0.1057, 0.1030, 0.0973, 0.1058, 0

In [663]:
_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 2.1575, Valid Accuracy: 0.7970
best model updated
Epoch 2, Train Loss: 1.9009, Valid Accuracy: 0.8913
best model updated
Epoch 3, Train Loss: 1.7746, Valid Accuracy: 0.9190
best model updated
Epoch 4, Train Loss: 1.6907, Valid Accuracy: 0.9390
best model updated
Epoch 5, Train Loss: 1.6342, Valid Accuracy: 0.9450
best model updated
Epoch 6, Train Loss: 1.5953, Valid Accuracy: 0.9590
best model updated
Epoch 7, Train Loss: 1.5656, Valid Accuracy: 0.9633
Epoch 8, Train Loss: 1.5450, Valid Accuracy: 0.9620
best model updated
Epoch 9, Train Loss: 1.5288, Valid Accuracy: 0.9660
best model updated
Epoch 10, Train Loss: 1.5174, Valid Accuracy: 0.9670
Epoch 11, Train Loss: 1.5071, Valid Accuracy: 0.9630
Epoch 12, Train Loss: 1.4999, Valid Accuracy: 0.9607
best model updated
Epoch 13, Train Loss: 1.4946, Valid Accuracy: 0.9683
Epoch 14, Train Loss: 1.4895, Valid Accuracy: 0.9670
best model updated
Epoch 15, Train Loss: 1.4857, Valid Accuracy: 0.9690
Epoch

5-5. Multi-Label Classification (with 4 Labels)

In [664]:
# Multi-Label Classification 용으로 변경

from torch.utils.data import Dataset

class MNIST_mlc(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.class_map = {0: torch.tensor([1., 0., 1., 1.]),
                          1: torch.tensor([0., 0., 0., 1.]),
                          2: torch.tensor([1., 1., 0., 0.]),
                          3: torch.tensor([0., 1., 1., 0.]),
                          4: torch.tensor([1., 0., 0., 1.]),
                          5: torch.tensor([0., 1., 0., 0.]),
                          6: torch.tensor([1., 0., 1., 0.]),
                          7: torch.tensor([0., 1., 0., 0.]),
                          8: torch.tensor([1., 0., 1., 0.]),
                          9: torch.tensor([0., 0., 1., 1.])}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        new_label = self.class_map[label]
        return image, new_label

In [665]:
# Multi-Label Classification 용 Data Loader

train_loader_mlc = DataLoader(MNIST_mlc(train_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=True)

valid_loader_mlc = DataLoader(MNIST_mlc(valid_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=False)

test_loader_mlc = DataLoader(MNIST_mlc(test_loader.dataset),
                            batch_size=BATCH_SIZE,
                            shuffle=False)

In [666]:
# 데이터셋 클래스 분포 분석 함수

def analyze_class_distribution_mlc(data_loader):
    targets = [str(data_loader.dataset[i][1]) for i in range(len(data_loader.dataset))]

    class_list, class_counts = np.unique(targets, return_counts=True)
    class_percentage = np.array(class_counts) * 100.0 / sum(class_counts)

    class_distrib = pd.DataFrame({'class': class_list,
                                  'count': class_counts,
                                  'percentage (%)': class_percentage})

    return class_distrib

In [667]:
analyze_class_distribution_mlc(train_loader_mlc)

,class,count,percentage (%)
0,"tensor([0., 0., 0., 1.])",239,11.95
1,"tensor([0., 0., 1., 1.])",211,10.55
2,"tensor([0., 1., 0., 0.])",372,18.60
3,"tensor([0., 1., 1., 0.])",214,10.70
4,"tensor([1., 0., 0., 1.])",200,10.00
5,"tensor([1., 0., 1., 0.])",382,19.10
6,"tensor([1., 0., 1., 1.])",186,9.30
7,"tensor([1., 1., 0., 0.])",196,9.80


In [668]:
analyze_class_distribution_mlc(valid_loader_mlc)

,class,count,percentage (%)
0,"tensor([0., 0., 0., 1.])",316,10.533333
1,"tensor([0., 0., 1., 1.])",296,9.866667
2,"tensor([0., 1., 0., 0.])",542,18.066667
3,"tensor([0., 1., 1., 0.])",331,11.033333
4,"tensor([1., 0., 0., 1.])",305,10.166667
5,"tensor([1., 0., 1., 0.])",581,19.366667
6,"tensor([1., 0., 1., 1.])",288,9.600000
7,"tensor([1., 1., 0., 0.])",341,11.366667


In [669]:
analyze_class_distribution_mlc(test_loader_mlc)

,class,count,percentage (%)
0,"tensor([0., 0., 0., 1.])",1135,11.35
1,"tensor([0., 0., 1., 1.])",1009,10.09
2,"tensor([0., 1., 0., 0.])",1920,19.20
3,"tensor([0., 1., 1., 0.])",1010,10.10
4,"tensor([1., 0., 0., 1.])",982,9.82
5,"tensor([1., 0., 1., 0.])",1932,19.32
6,"tensor([1., 0., 1., 1.])",980,9.80
7,"tensor([1., 1., 0., 0.])",1032,10.32


In [670]:
# 정상적으로 Binary Cross-Entropy 적용 시

# Sigmoid 를 이미 적용하여 최종 출력값을 0 ~ 1 로 만들었으므로,
# nn.BCEWithLogitsLoss() 가 아닌 nn.BCELoss() 를 사용

_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_mlc,
                                     valid_loader=valid_loader_mlc,
                                     test_loader=test_loader_mlc,
                                     num_classes=4,
                                     final_activation=nn.Sigmoid(),
                                     multi_label=True)

best model updated
Epoch 1, Train Loss: 0.6004, Valid Accuracy: 0.7886
best model updated
Epoch 2, Train Loss: 0.3887, Valid Accuracy: 0.9075
best model updated
Epoch 3, Train Loss: 0.2616, Valid Accuracy: 0.9511
best model updated
Epoch 4, Train Loss: 0.1853, Valid Accuracy: 0.9671
best model updated
Epoch 5, Train Loss: 0.1363, Valid Accuracy: 0.9704
best model updated
Epoch 6, Train Loss: 0.1075, Valid Accuracy: 0.9762
Epoch 7, Train Loss: 0.0856, Valid Accuracy: 0.9702
Epoch 8, Train Loss: 0.0677, Valid Accuracy: 0.9747
best model updated
Epoch 9, Train Loss: 0.0545, Valid Accuracy: 0.9778
Epoch 10, Train Loss: 0.0516, Valid Accuracy: 0.9719
best model updated
Epoch 11, Train Loss: 0.0402, Valid Accuracy: 0.9786
best model updated
Epoch 12, Train Loss: 0.0320, Valid Accuracy: 0.9794
best model updated
Epoch 13, Train Loss: 0.0273, Valid Accuracy: 0.9797
best model updated
Epoch 14, Train Loss: 0.0240, Valid Accuracy: 0.9800
best model updated
Epoch 15, Train Loss: 0.0199, Valid Acc

In [671]:
# Softmax 함수 (with Binary Cross-Entropy) 적용 시

_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_mlc,
                                     valid_loader=valid_loader_mlc,
                                     test_loader=test_loader_mlc,
                                     num_classes=4,
                                     final_activation=nn.Softmax(),
                                     multi_label=True)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 0.7020, Valid Accuracy: 0.6598
best model updated
Epoch 2, Train Loss: 0.5106, Valid Accuracy: 0.6796
best model updated
Epoch 3, Train Loss: 0.4287, Valid Accuracy: 0.6968
best model updated
Epoch 4, Train Loss: 0.3830, Valid Accuracy: 0.7021
best model updated
Epoch 5, Train Loss: 0.3569, Valid Accuracy: 0.7077
best model updated
Epoch 6, Train Loss: 0.3448, Valid Accuracy: 0.7083
best model updated
Epoch 7, Train Loss: 0.3272, Valid Accuracy: 0.7296
best model updated
Epoch 8, Train Loss: 0.3200, Valid Accuracy: 0.7366
best model updated
Epoch 9, Train Loss: 0.3139, Valid Accuracy: 0.7438
best model updated
Epoch 10, Train Loss: 0.3115, Valid Accuracy: 0.7443
Epoch 11, Train Loss: 0.3072, Valid Accuracy: 0.7436
best model updated
Epoch 12, Train Loss: 0.3047, Valid Accuracy: 0.7462
best model updated
Epoch 13, Train Loss: 0.3018, Valid Accuracy: 0.7542
Epoch 14, Train Loss: 0.2960, Valid Accuracy: 0.7538
Epoch 15, Train Loss: 0.2942, Valid Acc

In [672]:
# Softmax 함수 (with Categorical Cross-Entropy) 적용 시

_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader_mlc,
                                     valid_loader=valid_loader_mlc,
                                     test_loader=test_loader_mlc,
                                     num_classes=4,
                                     final_activation=nn.Softmax(),
                                     multi_label=True)

best model updated
Epoch 1, Train Loss: 2.3574, Valid Accuracy: 0.7056
best model updated
Epoch 2, Train Loss: 2.1987, Valid Accuracy: 0.7345
best model updated
Epoch 3, Train Loss: 2.1346, Valid Accuracy: 0.7572
Epoch 4, Train Loss: 2.0952, Valid Accuracy: 0.7486
best model updated
Epoch 5, Train Loss: 2.0712, Valid Accuracy: 0.7603
best model updated
Epoch 6, Train Loss: 2.0531, Valid Accuracy: 0.7625
best model updated
Epoch 7, Train Loss: 2.0413, Valid Accuracy: 0.7742
Epoch 8, Train Loss: 2.0342, Valid Accuracy: 0.7709
best model updated
Epoch 9, Train Loss: 2.0281, Valid Accuracy: 0.7774
Epoch 10, Train Loss: 2.0203, Valid Accuracy: 0.7717
best model updated
Epoch 11, Train Loss: 2.0206, Valid Accuracy: 0.7798
Epoch 12, Train Loss: 2.0186, Valid Accuracy: 0.7746
best model updated
Epoch 13, Train Loss: 2.0172, Valid Accuracy: 0.7809
Epoch 14, Train Loss: 2.0152, Valid Accuracy: 0.7750
best model updated
Epoch 15, Train Loss: 2.0089, Valid Accuracy: 0.7819
Epoch 16, Train Loss: 2.